In [1]:
import sys
sys.path.insert(0,"../")

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
from core.dataloader import CLEVR_train, collate_boxes
from torch.utils.data import Dataset, DataLoader

ImportError: /lib64/libc.so.6: version `GLIBC_2.14' not found (required by /home/mprabhud/miniconda3/envs/disco/lib/python3.7/site-packages/torch/_C.cpython-37m-x86_64-linux-gnu.so)

In [ ]:
train_dataset = CLEVR_train(root_dir='/home/mprabhud/dataset/clevr_lang/npys/ab_5t.txt')
train_loader = DataLoader(train_dataset, batch_size=5, shuffle=True, collate_fn=collate_boxes)

In [ ]:
for b in train_loader:
    feed_dict_q, feed_dict_k, metadata = b
    break

In [ ]:
feed_dict_q["images"] = feed_dict_k["images"].cuda()
feed_dict_k["images"] = feed_dict_k["images"].cuda()

In [ ]:
# default_exp model.scene_graph.scene_graph

In [ ]:
#export

import torch
import torch.nn as nn
from torchvision.ops import RoIAlign
from torchvision.models import resnet34

from core.model.scene_graph.utils import *

class SceneGraph(nn.Module):
    def __init__(self, feature_dim=256, output_dims=[256,256], downsample_rate=16, mode=None):
        super().__init__()
        self.pool_size = 7
        self.feature_dim = feature_dim
        self.output_dims = output_dims
        self.downsample_rate = downsample_rate
        self.mode = mode
        
        self.object_roi_pool = RoIAlign(self.pool_size, 1.0 / self.downsample_rate, -1)
#         self.context_roi_pool = RoIAlign(self.pool_size, 1.0 / self.downsample_rate, -1)
        self.relation_roi_pool = RoIAlign(self.pool_size, 1.0 / self.downsample_rate, -1)
        
#         self.context_feature_extract = nn.Conv2d(feature_dim, feature_dim, 1)
        self.relation_feature_extract = nn.Conv2d(feature_dim, feature_dim // 2 * 3, 1)

#         self.object_feature_fuse = nn.Conv2d(feature_dim * 2, output_dims[0], 1)
        self.relation_feature_fuse = nn.Conv2d(feature_dim // 2 * 3 + output_dims[0] * 2, output_dims[1], 1)

        self.object_feature_fc = nn.Sequential(nn.ReLU(True), nn.Linear(output_dims[0] * self.pool_size ** 2, output_dims[0]))
        self.relation_feature_fc = nn.Sequential(nn.ReLU(True), nn.Linear(output_dims[1] * self.pool_size ** 2, output_dims[1]))
        
        # this will change for models with multiple objects in future
        # in that case, it will pick up the pretrained weights
        if mode=="node":
            self.reset_parameters()
            
#         if self.mode=="spatial":
#             print("freezing feature extractor scene graph")
#             self.set_parameter_requires_grad()
            
    def set_parameter_requires_grad(self):
        self.object_roi_pool.requires_grad = False
        self.object_feature_fc.requires_grad = False
        
    def reset_parameters(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight.data)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight.data)
                m.bias.data.zero_()
                
    def forward(self, image_features, objects, objects_length):
        
        mode = self.mode
        object_features = image_features
#         context_features = self.context_feature_extract(image_features)
        relation_features = self.relation_feature_extract(image_features)

        outputs = list()
        objects_index = 0
        for i in range(image_features.size(0)):
            box = objects[objects_index:objects_index + objects_length[i].item()]
            objects_index += objects_length[i].item()

            with torch.no_grad():
                batch_ind = i + torch.zeros(box.size(0), 1, dtype=box.dtype, device=box.device)

                # generate a "full-image" bounding box
                image_h, image_w = image_features.size(2) * self.downsample_rate, image_features.size(3) * self.downsample_rate
                image_box = torch.cat([
                    torch.zeros(box.size(0), 1, dtype=box.dtype, device=box.device),
                    torch.zeros(box.size(0), 1, dtype=box.dtype, device=box.device),
                    image_w + torch.zeros(box.size(0), 1, dtype=box.dtype, device=box.device),
                    image_h + torch.zeros(box.size(0), 1, dtype=box.dtype, device=box.device)
                ], dim=-1)

                # meshgrid to obtain the subject and object bounding boxes
                sub_id, obj_id = meshgrid(torch.arange(box.size(0), dtype=torch.int64, device=box.device), dim=0)
                sub_id, obj_id = sub_id.contiguous().view(-1), obj_id.contiguous().view(-1)
                sub_box, obj_box = meshgrid(box, dim=0)
                sub_box = sub_box.contiguous().view(box.size(0) ** 2, 4)
                obj_box = obj_box.contiguous().view(box.size(0) ** 2, 4)

                # union box
                union_box = generate_union_box(sub_box, obj_box)
                rel_batch_ind = i + torch.zeros(union_box.size(0), 1, dtype=box.dtype, device=box.device)

                # intersection maps
#                 box_context_imap = generate_intersection_map(box, image_box, self.pool_size)
                sub_union_imap = generate_intersection_map(sub_box, union_box, self.pool_size)
                obj_union_imap = generate_intersection_map(obj_box, union_box, self.pool_size)

#             this_context_features = self.context_roi_pool(context_features, torch.cat([batch_ind, image_box], dim=-1))
#             x, y = this_context_features.chunk(2, dim=1)
#             this_object_features = self.object_feature_fuse(torch.cat([
#                 self.object_roi_pool(object_features, torch.cat([batch_ind, box], dim=-1)),
#                 x, y * box_context_imap
#             ], dim=1))
            
            this_object_features = self.object_roi_pool(object_features, torch.cat([batch_ind, box], dim=-1))
            
            if mode=="node":
                outputs.append([
                    self._norm(self.object_feature_fc(this_object_features.view(box.size(0), -1))), None
                ])                
            elif mode=="spatial":
                this_relation_features = self.relation_roi_pool(relation_features, torch.cat([rel_batch_ind, union_box], dim=-1))
                x, y, z = this_relation_features.chunk(3, dim=1)
                this_relation_features = self.relation_feature_fuse(torch.cat([
                    this_object_features[sub_id], this_object_features[obj_id],
                    x, y * sub_union_imap, z * obj_union_imap
                ], dim=1))

                outputs.append([
                    self._norm(self.object_feature_fc(this_object_features.view(box.size(0), -1))),
                    self._norm(self.relation_feature_fc(this_relation_features.view(box.size(0) * box.size(0), -1)).view(box.size(0), box.size(0), -1))
                ])
            else:
                raise ValueError("Feature Generation mode not defined properly. It should be either 'node' or 'spatial'.")

        return outputs

    def _norm(self, x):
        return x / x.norm(2, dim=-1, keepdim=True)  

In [ ]:
resnet = resnet34(pretrained=True)
feature_extractor = nn.Sequential(*list(resnet.children())[:-3])
feature_extractor =  feature_extractor.cuda()

In [ ]:
scene_graph = SceneGraph(feature_dim=256, 
                         output_dims=[256,256],
                         downsample_rate=16,
                         mode="node")
scene_graph = scene_graph.cuda()

In [ ]:
image_features = feature_extractor(feed_dict_q["images"])
outputs = scene_graph(image_features, feed_dict_q["objects_boxes"], feed_dict_q["objects"])

In [ ]:
outputs[0][0].shape

In [ ]:
outputs[0][1]

In [ ]:
scene_graph = SceneGraph(feature_dim=256, 
                         output_dims=[256,256],
                         downsample_rate=16,
                         mode="spatial")
scene_graph = scene_graph.cuda()

In [ ]:
image_features = feature_extractor(feed_dict_q["images"])
outputs = scene_graph(image_features, feed_dict_q["objects_boxes"], feed_dict_q["objects"])

In [ ]:
outputs[0][0].shape, outputs[0][1].shape